In [1]:
# Dependencies
import pandas as pd
import requests
import time
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
import datetime as dt

In [2]:
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path)

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text.

In [3]:
# NASA Mars site to scrape
url_nasa = "https://mars.nasa.gov/news"

In [4]:
browser.visit(url_nasa)
time.sleep(1)

In [5]:
html = browser.html
soup = bs(html, "html.parser")

In [6]:
# NASA MARS top news story - title
news_title = soup.find('div', class_="content_title").text.strip()
news_title

'The MarCO Mission Comes to an End'

In [7]:
# NASA MARS top news story - paragraph
news_p = soup.find('div', class_="article_teaser_body").text.strip()
news_p

'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'

### JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

In [8]:
url_jpl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_jpl)
time.sleep(1)

In [9]:
#click the button
browser.find_by_id('full_image').first.click()
time.sleep(1)

In [10]:
html = browser.html
soup = bs(html, "html.parser")

In [11]:
marsimage = soup.find('img', class_="fancybox-image").get("src")
baseurl = 'https://www.jpl.nasa.gov'
featured_image_url = baseurl+marsimage
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17474_ip.jpg'

### Mars Weather

* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

In [12]:
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)
time.sleep(3)

In [13]:
html = browser.html
soup = bs(html, "html.parser")

response = requests.get(url_weather)
time.sleep(1)

In [14]:
twitter_soup = bs(response.text, 'html.parser')
timeline = twitter_soup.select('#timeline li.stream-item')

In [15]:
all_tweets = []
for tweet in timeline:
    tweet_text = tweet.select('p.tweet-text')[0].get_text()
    all_tweets.append({"text": tweet_text})
mars_weather = all_tweets[0]
print(mars_weather)

{'text': '360 Sol 2687 selfiehttps://roundme.com/tour/550044/view?welcome\xa0…'}


### Mars Facts

* Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [16]:
url_facts = "https://space-facts.com/mars"
browser.visit(url_facts)
time.sleep(1)

In [17]:
fact_df_1 = pd.read_html(url_facts)
url_facts_df = fact_df_1[0]
url_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [18]:
url_facts_df2 = url_facts_df.set_index(0).rename(columns={1:"Value"})
url_facts_df2

,Value
0,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [19]:
url_facts_html = url_facts_df2.to_html(classes="table-condensed")
url_facts_html

'<table border="1" class="dataframe table-condensed">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>0</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    

### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

In [20]:
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemispheres)
time.sleep(1)

In [21]:
hemispheres_data =[]

for i in range (4):
    browser.find_by_css('a.product-item h3')[i].click()
    time.sleep(2)
    html = browser.html
    soup = bs(html, "html.parser")
    heminame = soup.find('h2').text.strip()
    hemipic = soup.find('a', target="_blank").get("href")
    hemisphere = {
        "title" : heminame,
        "img_url" : hemipic
    }
    hemispheres_data.append(hemisphere)
    browser.back()
hemispheres_data

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]